In [1]:
# For dialog box interface
import tkinter as tk
from tkinter import ttk
from tkinter import filedialog as fd
from tkinter.messagebox import showinfo
import os

# For overbooking optimization monte carlo simulation
import shutil  # For copying the input excel

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import openpyxl
import notebook

from datetime import date #for naming output file

def monte_carlo_no_show(trials, A, distribution='negative_binomial', mean=None, std=None, historical_no_show_vector=None):
    """
    Simulate the number of no-show events using Monte Carlo simulation.

    This function generates simulated counts of no-show events based on specified statistical distributions.
    It allows for various distribution types and can accommodate parameters necessary for those distributions.

    Parameters:
    - trials (int): The number of Monte Carlo trials to conduct.
    - A (int): The total number of discrete events for which no-show counts are simulated.
    - distribution (str): The type of statistical distribution to use for generating no-show counts.
        Options include:
        - 'negative_binomial': Models the number of failures until a fixed number of successes occurs.
        - 'binomial': Models the number of successes in a fixed number of trials.
        - 'normal': Generates values based on a normal distribution.
        - 'historical_pull_with_replacement': Uses historical no-show data to simulate counts.
    - mean (float, optional): The mean value for distributions that require it (e.g., normal distribution).
    - std (float, optional): The standard deviation for distributions that require it (e.g., normal distribution).
    - historical_no_show_vector (array-like, optional): An array of historical no-show percentages used when
      'historical_pull_with_replacement' distribution is selected.

    Returns:
    - no_show_counts (numpy.ndarray): An array of simulated no-show counts generated from each trial.

    Raises:
    - ValueError: If an unsupported distribution type is specified.
    """

    if distribution == 'negative_binomial':
        # Calculate parameters based on mean and std
        p = mean/(std**2) # Probability of success
        n = (mean**2) / (std**2 - mean) # Number of successes

        N = np.random.negative_binomial(n=n, p=p, size=trials)
    elif distribution == 'normal':
        N = np.random.normal(loc=mean, scale=std, size=trials)
        N = np.maximum(0, N).astype(int)
    elif distribution == 'historical_pull_with_replacement':
        ns_percentage = np.random.choice(historical_no_show_vector, size=trials, replace=True)
        N = np.round(ns_percentage * A, 0)
    else:
        raise ValueError("Unsupported distribution type")

    return N

def calculate_value(results, CDB, OC, C, A):
    """
    Calculate the average value based on no-show counts and additional cost parameters.

    This function computes the average value by applying a formula that considers:
    - The operational cost (OC) for each event.
    - The cost of no-shows (CDB) based on the difference between the total events (A)
      and the actual no-show counts.

    Parameters:
    - results (list or numpy.ndarray): A list or array of no-show counts, typically obtained
      from a Monte Carlo simulation.
    - CDB (float): The cost incurred per no-show event, representing the financial impact
      of no-shows.
    - OC (float): The operational cost incurred for each event, representing the baseline cost
      for conducting the event.
    - C (int): The number of events that were conducted.
    - A (int): The total number of discrete events scheduled.

    Returns:
    - average_value (float): The calculated average value based on the provided formula:
      average_value = mean(OC * (A - C) - CDB * max(A - C - results, 0))

    Example:
    >>> no_show_counts = [5, 10, 15]  # Example no-show counts
    >>> average_val = calculate_value(no_show_counts, CDB, OC, C, A)
    """

    return np.mean(OC * (A - C) - CDB * np.maximum(A - C - np.array(results), 0))

def get_value_from_settings(trials, A, distribution, NS, NS_std, CDB, OC, C, historical_no_show_vector=None):
    """
    Calculate the monetary value based on simulated no-show events and cost parameters.

    This function utilizes Monte Carlo simulation to estimate the number of no-show events
    and subsequently calculates the associated monetary value considering operational costs
    and costs per no-show.

    Parameters:
    - trials (int): The number of Monte Carlo trials to conduct for simulating no-show events.
    - A (int): The total number of discrete events for which no-show counts are simulated.
    - distribution (str): The type of statistical distribution to use for generating no-show counts.
    - NS (float): The average no-show rate, which is scaled by the total events (A) to determine the mean.
    - NS_std (float): The standard deviation of the no-show rate, scaled by the total events (A).
    - CDB (float): The cost incurred per no-show event, representing the financial impact of no-shows.
    - OC (float): The operational cost incurred for each event, representing the baseline cost for conducting the event.
    - C (int): The number of events that were conducted.
    - historical_no_show_vector (array-like, optional): An array of historical no-show percentages used when
      a distribution that requires historical data is selected.

    Returns:
    - monetary_value (float): The calculated monetary value based on the simulated no-show counts and cost parameters.

    Example:
    >>> value = get_value_from_settings(trials=1000, A=100, distribution='negative_binomial',
    ... NS=0.1, NS_std=0.05, CDB=50, OC=20, C=90)
    """

    simulated_values = monte_carlo_no_show(trials, A, distribution, mean=NS*A, std=NS_std*A, historical_no_show_vector=historical_no_show_vector)
    monetary_value = calculate_value(simulated_values,CDB, OC, C, A)

    return monetary_value

def run_optimal_overbooking_finder(
    trials,
    NS,
    NS_std,
    CDB,
    OC,
    C,
    distribution_c=1000,
    Leg="",
    current_lid_percentage=1.0,
    models = ['negative_binomial', 'normal', 'historical_pull_with_replacement'],
    ns_vector = None
    ):
    """
    Find the optimal overbooking level based on simulated no-show events and associated costs.

    This function runs simulations to determine the optimal overbooking level by evaluating
    various statistical models for no-show events. It calculates expected values for different
    overbooking scenarios and plots the results.

    Parameters:
    - trials (int): The number of Monte Carlo trials to conduct for simulating no-show events.
    - NS (float): The average no-show rate, which is used to determine the mean for simulations.
    - NS_std (float): The standard deviation of the no-show rate, influencing the variability in simulations.
    - CDB (float): The cost incurred per no-show event, representing the financial impact of no-shows.
    - OC (float): The operational cost incurred for each event, representing the baseline cost for conducting the event.
    - C (int): The number of events that were conducted.
    - distribution_c (int, optional): The base number of events for calculating overbooking levels (default is 1000).
    - Leg (str, optional): An identifier for the overbooking distribution (default is an empty string).
    - current_lid_percentage (float, optional): The current lid percentage to compare against optimal values (default is 1.0).
    - models (list of str, optional): A list of statistical models to evaluate for no-show events (default includes 'negative_binomial', 'normal', and 'historical_pull_with_replacement').
    - ns_vector (array-like, optional): An array of historical no-show percentages used when a distribution that requires historical data is selected.

    Returns:
    - results_dict (dict): A dictionary containing the optimal overbooking values and corresponding lid percentages for each distribution model.

    Example:
    >>> optimal_results = run_optimal_overbooking_finder(trials=1000, NS=0.1, NS_std=0.05, CDB=50, OC=20, C=90, models = ['negative_binomial'])
    """

    # Loop over the results
    max_overbooking_level = 0.09

    raw_c = C

    A_vector = [i for i in range(distribution_c, round(distribution_c*(1+max_overbooking_level)+1))]

    result_list = []
    for act_A in A_vector:
        act_results = {
            "Lid": act_A * raw_c/distribution_c
        }

        for act_distribution in models:
            try:
                act_std = NS_std * act_A
                act_mu = NS * act_A
                act_value = get_value_from_settings(
                    trials,
                    act_A,
                    act_distribution,
                    NS,
                    NS_std,
                    CDB,
                    OC,
                    distribution_c,
                    historical_no_show_vector=ns_vector
                )

                act_value = act_value * raw_c/distribution_c

                act_results[act_distribution] = act_value
            except:
                print("Didn't add distribution: {}".format(act_distribution))

            result_list.append(act_results)


    result_df = pd.DataFrame(result_list)

    # Format result
    results_dict = {}

    # Annotate the maximum values for each series
    for column in result_df.columns[1:]:
        max_value = result_df[column].max()
        max_index = result_df[column].idxmax()
        max_x_value = result_df["Lid"].iloc[max_index]

        lid_percentage = round(max_x_value/C, 3)

        # Get the current value
        current_lid_abs = current_lid_percentage*distribution_c
        current_lid_abs_scaled = current_lid_abs * raw_c/distribution_c
        current_value = get_value_from_settings(
            trials,
            round(current_lid_abs,0),
            column,
            NS,
            NS_std,
            CDB,
            OC,
            distribution_c,
            historical_no_show_vector=ns_vector
        )

        current_value = current_value * raw_c/distribution_c
        results_dict["current_overbooking_value_{}".format(column)] = current_value

        results_dict[column] = lid_percentage
        results_dict["optimal_overbooking_value_{}".format(column)] = max_value

    return results_dict

def input_file():
    """
    Generates a GUI for the whole program
    """
    filetypes = (
        ('xlsx files', '*.xlsx'),
        ('All files', '*.*')
    )

    # Get the current working directory
    current_directory = os.getcwd()

    input_filename = fd.askopenfilename(
        title='Open excel file',
        initialdir=current_directory,
        filetypes=filetypes)

    showinfo(
        title='Selected File',
        message=input_filename
    )

    # Create output filename based on current date
    current_date = date.today()
    output_filename = f'{current_date}{'_OB.xlsx'}'

    # Step 1: Copy the input excel file and save it as the name of the output file
    shutil.copy(input_filename, output_filename)

    # Step 2: Read the sheets into pandas dataframes
    # Skip the first two rows and the first column
    loop_df = pd.read_excel(input_filename, sheet_name='loop_df', skiprows=0)
    raw_actual_ns_vector_df = pd.read_excel(input_filename, sheet_name='raw_ns_data', skiprows=0)

    # Setting the number of simulations (higher creates a more stable outcome)
    trials = 100000  # Number of trials, to review how many to use given time complexity O and accuracy tradeoff

    for i in range(len(loop_df)):

      Leg = loop_df.loc[i, "Leg"]
      Airline = loop_df.loc[i, "Airline Code"]
      C = loop_df.loc[i, "Avg. Capacity"]

      NS = loop_df.loc[i, "mu"]
      NS_std = loop_df.loc[i, "sigma"]

      CDB = loop_df.loc[i, "Cost of denied boarding [PHP]"]
      OC = loop_df.loc[i, "Cost of empty seat [PHP]"]

      current_lid_percentage = loop_df.loc[i, "Current Lid %"]

      # get current vector of no_show_data
      act_ns_vector_df = raw_actual_ns_vector_df.loc[raw_actual_ns_vector_df["Leg"] == Leg]
      if not pd.isna(Airline):
          act_ns_vector_df = act_ns_vector_df.loc[act_ns_vector_df["Airline Code"] == Airline]
      act_ns_vector = act_ns_vector_df["NS%"].values

      act_results = run_optimal_overbooking_finder(
          trials,
          NS,
          NS_std,
          CDB,
          OC,
          C,
          Leg=Leg,
          current_lid_percentage=current_lid_percentage,
          ns_vector = act_ns_vector
      )

      for act_key, act_value in act_results.items():
          loop_df.loc[i, act_key] = act_value

    # Write results to "sheet 3" in the copied Excel file
    with pd.ExcelWriter(output_filename, engine='openpyxl', mode='a') as writer:
      loop_df.to_excel(writer, sheet_name='results', index=False)
      print("Data has been written to 'results' in {}.".format(output_filename))

    showinfo(
        title='Output file generated',
        message=f"Data has been written to 'results' in {output_filename}."
    )

    root.destroy()

if __name__ == "__main__":
  # For reproducibility
  np.random.seed(29)

  # Dialog box interface
  # create the root window
  root = tk.Tk()
  root.title('Overbooking Optimization')
  root.resizable(False, False)
  root.geometry('300x150')

  # open button
  open_button = ttk.Button(
      root,
      text='Select input file',
      command=input_file
  )

  open_button.pack(expand=True)

  # run the application
  root.mainloop()


Data has been written to 'results' in 2025-11-18_OB.xlsx.


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\dladaenielle\AppData\Local\Programs\Python\Python313\Lib\tkinter\__init__.py", line 2074, in __call__
    return self.func(*args)
           ~~~~~~~~~^^^^^^^
  File "C:\Users\dladaenielle\AppData\Local\Temp\ipykernel_26608\866059728.py", line 319, in input_file
    showinfo(
    ~~~~~~~~^
        title='Output file generated',
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        message=f"Data has been written to 'results' in {output_filename}."
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    )
    ^
  File "C:\Users\dladaenielle\AppData\Local\Programs\Python\Python313\Lib\tkinter\messagebox.py", line 88, in showinfo
    return _show(title, message, INFO, OK, **options)
  File "C:\Users\dladaenielle\AppData\Local\Programs\Python\Python313\Lib\tkinter\messagebox.py", line 76, in _show
    res = Message(**options).show()
  File "C:\Users\dladaenielle\AppData\Local\Programs\Python\P